In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Import Data
data = pd.read_csv("/Users/edilbekabdyrakhmanov/Documents/GitHub/bakeryy/0_DataPreparation/initialdata/merged_data_temperature+holidays+weather_impressions.csv")
data.head()  # Print first few rows to verify

,Datum,Weekday,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,id,Warengruppe,Umsatz,KielerWoche,AverageTemp,Temp_Deviation,Temp_vs_Avg,Weather_Impression,Is_Holiday
0,2012-01-01,Sunday,8.0,9.82,14,58.0,NaN,NaN,NaN,NaN,6.504342,3.315658,Significantly warmer,okay,1
1,2012-01-02,Monday,7.0,7.44,12,0.0,NaN,NaN,NaN,NaN,3.062303,4.377697,Significantly warmer,very good,0
2,2012-01-03,Tuesday,8.0,5.54,18,63.0,NaN,NaN,NaN,NaN,3.062303,2.477697,Significantly warmer,okay,0
3,2012-01-04,Wednesday,4.0,5.69,19,80.0,NaN,NaN,NaN,NaN,3.062303,2.627697,Significantly warmer,very bad,0
4,2012-01-05,Thursday,6.0,5.30,23,80.0,NaN,NaN,NaN,NaN,3.062303,2.237697,Significantly warmer,very bad,0


DATA PREPARATION

In [8]:
# Define categorical features
categorical_features = ['Weekday', 'Warengruppe', 'Weather_Impression', 'Temp_vs_Avg', 'KielerWoche', 'Is_Holiday']
numerical_features = ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']

# Inspect data types and unique values for categorical columns
print(data[categorical_features].dtypes)
print("Unique Values:\n", data[categorical_features].apply(lambda x: x.unique()))

# Ensure categorical columns are treated as categories
for col in categorical_features:
    data[col] = data[col].astype('category')

# Encode categorical variables using pd.get_dummies
features = pd.get_dummies(data[categorical_features], drop_first=True, dtype=int)

# Include numerical features and date column
for col in numerical_features:
    features[col] = data[col]
features['Datum'] = data['Datum']  # Keep the date column

# Construct the prepared data set including the dependent variable ('Umsatz')
prepared_data = pd.concat([data[['Umsatz']], features], axis=1)

# Handle missing values by removing rows with any missing values
prepared_data = prepared_data.dropna()

# Display the shape of the prepared data set
print(prepared_data.shape)
prepared_data.head()

Weekday               category
Warengruppe           category
Weather_Impression    category
Temp_vs_Avg             object
KielerWoche           category
Is_Holiday            category
dtype: object
Unique Values:
 Weekday               ['Sunday', 'Monday', 'Tuesday', 'Wednesday', '...
Warengruppe           [NaN, 'Brot', 'Brötchen', 'Croissant', 'Kondit...
Weather_Impression    ['okay', 'very good', 'very bad', 'bad', 'good...
Temp_vs_Avg           [Significantly warmer, Normal, Slightly warmer...
KielerWoche                   [NaN, 1.0]
Categories (1, float64): [1.0]
Is_Holiday                         [1, 0]
Categories (2, int64): [0, 1]
dtype: object
(9264, 25)


,Umsatz,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Warengruppe_Brötchen,Warengruppe_Croissant,Warengruppe_Konditorei,...,Weather_Impression_very good,Temp_vs_Avg_Significantly colder,Temp_vs_Avg_Significantly warmer,Temp_vs_Avg_Slightly colder,Temp_vs_Avg_Slightly warmer,Is_Holiday_1,Bewoelkung,Temperatur,Windgeschwindigkeit,Datum
394,148.83,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,6.0,17.84,15,2013-07-01
395,535.86,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,6.0,17.84,15,2013-07-01
396,201.20,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,6.0,17.84,15,2013-07-01
397,65.89,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,6.0,17.84,15,2013-07-01
398,317.48,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,6.0,17.84,15,2013-07-01


Selection of Training, Validation and Test Data

In [9]:
# Convert date column to datetime if not already (assuming you have a 'Datum' column)
data['Datum'] = pd.to_datetime(data['Datum'])

# Set date ranges for splits
train_start = '2013-07-01'
train_end = '2017-07-31'
val_start = '2017-08-01'
val_end = '2018-07-31'
test_start = '2018-08-01'
test_end = '2019-07-30'

# Split the data based on date ranges
training_data = prepared_data[(prepared_data['Datum'] >= train_start) & (prepared_data['Datum'] <= train_end)]
validation_data = prepared_data[(prepared_data['Datum'] >= val_start) & (prepared_data['Datum'] <= val_end)]
test_data = prepared_data[(prepared_data['Datum'] >= test_start) & (prepared_data['Datum'] <= test_end)]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1)
test_features = test_data.drop('Umsatz', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)

Training features dimensions: (7487, 24)
Validation features dimensions: (1777, 24)
Test features dimensions: (0, 24)

Training labels dimensions: (7487, 1)
Validation labels dimensions: (1777, 1)
Test labels dimensions: (0, 1)


Data Export

In [12]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")